In [1]:
import tensorflow as tf
import tensorflow.keras as keras
import librosa
import librosa.feature
import glob
import numpy as np
from utils import load_data

## Data preprocessing

In [2]:
trainX, trainY, validationX, validationY, testX, testY = load_data(split_into_10s = True)

KeyboardInterrupt: 

In [9]:
def normalize_data(data, mu=None, std=None):
    if mu is None or std is None:
        mu = np.mean(data)
        std = np.std(data)
        
    norm_data = data.copy() - mu
    norm_data /= std
    return norm_data, mu, std

In [3]:
#Reshaping into (batch_size, height, width, channels)
trainX = trainX.reshape((trainX.shape[0], trainX.shape[1], trainX.shape[2], 1))
validationX = validationX.reshape((validationX.shape[0], validationX.shape[1], validationX.shape[2], 1))
testX = testX.reshape((testX.shape[0], testX.shape[1], testX.shape[2], 1))



In [10]:
trainX, mu, std = normalize_data(trainX)
validationX, _, _ = normalize_data(validationX, mu, std)
testX, _, _ = normalize_data(testX, mu, std)

In [4]:
trainX.shape
trainY.shape

(44936, 5)

## Basic CNN

In [5]:
indim_x = 128
indim_y = 106

In [11]:
CNN_model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(16, (3,3), input_shape=(indim_x, indim_y, 1)),
  tf.keras.layers.BatchNormalization(momentum=0.9),
  tf.keras.layers.Activation('relu'),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Dropout(0.25),
  tf.keras.layers.Conv2D(32, (3,3)),
  tf.keras.layers.BatchNormalization(momentum=0.9),
  tf.keras.layers.Activation('relu'),
  tf.keras.layers.MaxPooling2D(2,2), 
  tf.keras.layers.Dropout(0.25),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(100, activation='relu'),
  tf.keras.layers.Dense(5, activation='softmax')
])

In [12]:
CNN_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
CNN_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 126, 104, 16)      160       
_________________________________________________________________
batch_normalization_2 (Batch (None, 126, 104, 16)      64        
_________________________________________________________________
activation_2 (Activation)    (None, 126, 104, 16)      0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 63, 52, 16)        0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 63, 52, 16)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 61, 50, 32)        4640      
_________________________________________________________________
batch_normalization_3 (Batch (None, 61, 50, 32)       

In [13]:
CNN_model.fit(trainX, trainY, batch_size = 128, validation_data=(validationX, validationY), epochs=18)

Train on 44936 samples, validate on 5407 samples
Epoch 1/18
44936/44936 [==============================] - 22s 489us/sample - loss: 1.6097 - accuracy: 0.5346 - val_loss: 1.1932 - val_accuracy: 0.5707
Epoch 2/18
44936/44936 [==============================] - 20s 444us/sample - loss: 0.9106 - accuracy: 0.6613 - val_loss: 1.1110 - val_accuracy: 0.5757
Epoch 3/18
44936/44936 [==============================] - 20s 443us/sample - loss: 0.8257 - accuracy: 0.6935 - val_loss: 1.1520 - val_accuracy: 0.5872
Epoch 4/18
44936/44936 [==============================] - 20s 445us/sample - loss: 0.8005 - accuracy: 0.7056 - val_loss: 1.0237 - val_accuracy: 0.6442
Epoch 5/18
44936/44936 [==============================] - 20s 445us/sample - loss: 0.7647 - accuracy: 0.7182 - val_loss: 0.8758 - val_accuracy: 0.6519
Epoch 6/18
44936/44936 [==============================] - 20s 442us/sample - loss: 0.7277 - accuracy: 0.7306 - val_loss: 0.9405 - val_accuracy: 0.6625
Epoch 7/18
44936/44936 [=====================

In [37]:
CNN_model.evaluate(testX, testY, verbose=2)

5055/1 - 1s - loss: 0.8067 - accuracy: 0.7177


[0.8141717711730716, 0.71770525]